# FastF1 Telemetry

### This file contains scripts that serve as ETL footprints for the VisionF1 telemetry data pipeline. They obtain data from the API or the local cache, process it as needed, and the objective is to then upload it to the VisionF1 database (MongoDB Atlas).

## Race Pace

All data from 2018 to now.

In [ ]:
import fastf1
import pandas as pd
from datetime import datetime

SEASONS = range(2025, datetime.now().year + 1)

all_race_pace = []

for season in SEASONS:
    schedule = fastf1.get_event_schedule(season)
    schedule = schedule[schedule["EventFormat"] != "testing"]

    for _, event in schedule.iterrows():
        event_name = event["EventName"]
        round_number = event["RoundNumber"]
        try:
            session = fastf1.get_session(season, event_name, 'R')
            session.load()
        except Exception as e:
            print(f"Error loading session {season} {event_name}.")
            continue

        if not hasattr(session, '_laps'):
            print(f"No lap data for {season} {event_name}, skipping.")
            continue
        
        laps = session.laps
        laps = laps[laps['IsAccurate']]
        drivers = laps['Driver'].unique()

        driver_color_mapping = fastf1.plotting.get_driver_color_mapping(session=session)

        for driver in drivers:
            filtered_laps = laps.pick_drivers([driver])
            
            avg_laptime = filtered_laps["LapTime"].mean()
            std_laptime = filtered_laps["LapTime"].std()

            driver_info = session.get_driver(driver)
            driver_first_name = driver_info.get("FirstName", driver)
            driver_last_name = driver_info.get("LastName", "")
            driver_position = driver_info.get("Position", None)
            driver_color = driver_color_mapping.get(driver, "#cccccc")

            all_race_pace.append({
                "season": season,
                "round": int(round_number),
                "event": event_name,
                "driver": driver,
                "driver_first_name": driver_first_name,
                "driver_last_name": driver_last_name,
                "driver_position": driver_position,
                "driver_color": driver_color,
                "avg_laptime": avg_laptime.total_seconds(),
                "std_laptime": std_laptime.total_seconds() if pd.notnull(std_laptime) else None,
                "race_pace_id": f"{season}_{round_number}_{driver}"
            })

df_race_pace = pd.DataFrame(all_race_pace)

df_race_pace["race_pace_position"] = (
    df_race_pace.groupby(["season", "round"])["avg_laptime"]
    .rank(method="min", ascending=True)
    .astype(int)
)

In [86]:
df_race_pace

,season,round,event,driver,driver_first_name,driver_last_name,driver_position,driver_color,avg_laptime,std_laptime,race_pace_id,race_pace_position
0,2025,1,Australian Grand Prix,VER,Max,Verstappen,2.0,#0600ef,90.314777,2.618743,2025_1_VER,3
1,2025,1,Australian Grand Prix,GAS,Pierre,Gasly,11.0,#ff87bc,92.140342,2.566940,2025_1_GAS,9
2,2025,1,Australian Grand Prix,ANT,Andrea Kimi,Antonelli,4.0,#27f4d2,91.734382,2.871252,2025_1_ANT,6
3,2025,1,Australian Grand Prix,ALO,Fernando,Alonso,17.0,#00665f,92.502320,2.265482,2025_1_ALO,12
4,2025,1,Australian Grand Prix,LEC,Charles,Leclerc,8.0,#e80020,91.704388,2.618302,2025_1_LEC,5
...,...,...,...,...,...,...,...,...,...,...,...,...
360,2025,19,United States Grand Prix,SAI,Carlos,Sainz,20.0,#00a0dd,100.767250,0.411036,2025_19_SAI,17
361,2025,19,United States Grand Prix,HAD,Isack,Hadjar,16.0,#fcd700,100.788254,1.247889,2025_19_HAD,18
362,2025,19,United States Grand Prix,RUS,George,Russell,6.0,#27f4d2,99.807431,0.897683,2025_19_RUS,6
363,2025,19,United States Grand Prix,PIA,Oscar,Piastri,5.0,#ff8000,99.695700,0.823934,2025_19_PIA,5


In [87]:
ver = df_race_pace[ df_race_pace["driver"] == "VER" ]
ver_2025 = ver[ ver["season"] == 2025 ]
ver_2025

,season,round,event,driver,driver_first_name,driver_last_name,driver_position,driver_color,avg_laptime,std_laptime,race_pace_id,race_pace_position
0,2025,1,Australian Grand Prix,VER,Max,Verstappen,2.0,#0600ef,90.314777,2.618743,2025_1_VER,3
20,2025,2,Chinese Grand Prix,VER,Max,Verstappen,4.0,#0600ef,97.199452,1.050277,2025_2_VER,5
37,2025,3,Japanese Grand Prix,VER,Max,Verstappen,1.0,#0600ef,92.459200,0.927919,2025_3_VER,2
62,2025,4,Bahrain Grand Prix,VER,Max,Verstappen,6.0,#0600ef,98.083375,0.960517,2025_4_VER,5
78,2025,5,Saudi Arabian Grand Prix,VER,Max,Verstappen,2.0,#0600ef,93.191066,0.582681,2025_5_VER,1
95,2025,6,Miami Grand Prix,VER,Max,Verstappen,4.0,#0600ef,91.751291,1.136319,2025_6_VER,3
114,2025,7,Emilia Romagna Grand Prix,VER,Max,Verstappen,1.0,#0600ef,80.305176,1.168307,2025_7_VER,1
134,2025,8,Monaco Grand Prix,VER,Max,Verstappen,4.0,#0600ef,75.852114,1.453537,2025_8_VER,4
154,2025,9,Spanish Grand Prix,VER,Max,Verstappen,10.0,#0600ef,79.654830,1.071211,2025_9_VER,1
173,2025,10,Canadian Grand Prix,VER,Max,Verstappen,2.0,#0600ef,75.547278,0.688125,2025_10_VER,2


In [82]:
race_pace_2025 = df_race_pace[ df_race_pace["season"] == 2025 ]
race_pace_2025_15 = race_pace_2025[ race_pace_2025["round"] == 15 ]
race_pace_2025_15 = race_pace_2025_15.sort_values('avg_laptime')
dict = race_pace_2025_15.to_dict(orient="records")
print(dict)

[{'season': 2025, 'round': 15, 'event': 'Dutch Grand Prix', 'driver': 'PIA', 'driver_first_name': 'Oscar', 'driver_last_name': 'Piastri', 'driver_color': '#ff8000', 'avg_laptime': 74.48166, 'std_laptime': 0.98182, 'race_pace_id': '2025_15_PIA'}, {'season': 2025, 'round': 15, 'event': 'Dutch Grand Prix', 'driver': 'NOR', 'driver_first_name': 'Lando', 'driver_last_name': 'Norris', 'driver_color': '#ff8000', 'avg_laptime': 74.665288, 'std_laptime': 0.920907, 'race_pace_id': '2025_15_NOR'}, {'season': 2025, 'round': 15, 'event': 'Dutch Grand Prix', 'driver': 'VER', 'driver_first_name': 'Max', 'driver_last_name': 'Verstappen', 'driver_color': '#0600ef', 'avg_laptime': 75.104875, 'std_laptime': 1.047516, 'race_pace_id': '2025_15_VER'}, {'season': 2025, 'round': 15, 'event': 'Dutch Grand Prix', 'driver': 'HAD', 'driver_first_name': 'Isack', 'driver_last_name': 'Hadjar', 'driver_color': '#fcd700', 'avg_laptime': 75.201928, 'std_laptime': 0.968081, 'race_pace_id': '2025_15_HAD'}, {'season': 202

In [84]:
session = fastf1.get_session(2025, 'Dutch Grand Prix', 'R')
session.load()
fastf1.plotting.get_driver_color_mapping(session=session)
session.get_driver("COL")

core           INFO 	Loading data for Dutch Grand Prix - Race [v3.5.3]
2025-10-21 21:39:44,086 INFO Loading data for Dutch Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
2025-10-21 21:39:44,088 INFO Using cached data for session_info
req            INFO 	Using cached data for driver_info
2025-10-21 21:39:44,089 INFO Using cached data for driver_info
2025-10-21 21:39:44,096 DEBUG Failed to parse timestamp '-1:47:38.638' in Ergastresponse.
req            INFO 	Using cached data for session_status_data
2025-10-21 21:39:44,116 INFO Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
2025-10-21 21:39:44,118 INFO Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
2025-10-21 21:39:44,119 INFO Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
2025-10-21 21:39:44,201 INFO Using cached data for _extended_timing_data
req   

DriverNumber                              43
BroadcastName                    F COLAPINTO
Abbreviation                             COL
DriverId                           colapinto
TeamName                              Alpine
TeamColor                             00A1E8
TeamId                                alpine
FirstName                             Franco
LastName                           Colapinto
FullName                    Franco Colapinto
HeadshotUrl                             None
CountryCode                                 
Position                                11.0
ClassifiedPosition                        11
GridPosition                            16.0
Q1                                       NaT
Q2                                       NaT
Q3                                       NaT
Time                  0 days 00:00:14.511000
Status                              Finished
Points                                   0.0
Name: 43, dtype: object